<a href="https://colab.research.google.com/github/NdanyuzweP/A-CHAT-BOT/blob/main/A_Chat_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers tensorflow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, AdamWeightDecay
import pandas as pd
import numpy as np
import os

In [3]:
# Load dataset
file_path = "/content/CancerQA.csv"
df = pd.read_csv(file_path)

# Keep only relevant columns
df = df[['Question', 'Answer']]

# Combine question and answer into a conversational format
df['text'] = "Question: " + df['Question'] + " Answer: " + df['Answer']

# Display the first few rows
print(df.head())

# Save processed data for training
os.makedirs("/mnt/data", exist_ok=True)
df['text'].to_csv("/mnt/data/processed_dataset.csv", index=False)

                                            Question  \
0         What is (are) Non-Small Cell Lung Cancer ?   
1   Who is at risk for Non-Small Cell Lung Cancer? ?   
2  What are the symptoms of Non-Small Cell Lung C...   
3       How to diagnose Non-Small Cell Lung Cancer ?   
4  What is the outlook for Non-Small Cell Lung Ca...   

                                              Answer  \
0  Key Points\n                    - Non-small ce...   
1  Smoking is the major risk factor for non-small...   
2  Signs of non-small cell lung cancer include a ...   
3  Tests that examine the lungs are used to detec...   
4  Certain factors affect prognosis (chance of re...   

                                                text  
0  Question: What is (are) Non-Small Cell Lung Ca...  
1  Question: Who is at risk for Non-Small Cell Lu...  
2  Question: What are the symptoms of Non-Small C...  
3  Question: How to diagnose Non-Small Cell Lung ...  
4  Question: What is the outlook for Non-Small Ce..

In [4]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add special tokens if missing
tokenizer.pad_token = tokenizer.eos_token

# Load GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
# Load processed dataset
file_path = "/mnt/data/processed_dataset.csv"
df = pd.read_csv(file_path)

# Ensure 'text' column contains strings, not lists
df['text'] = df['text'].astype(str)

# Tokenize the text
inputs = tokenizer(list(df["text"]), max_length=512, truncation=True, padding="max_length", return_tensors="tf")

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((inputs["input_ids"], inputs["attention_mask"]))

# Function to map inputs and labels (labels = input_ids for GPT-2)
def map_to_dict(input_ids, attention_mask):
    return {"input_ids": input_ids, "attention_mask": attention_mask}, input_ids  # Labels = input_ids

train_dataset = train_dataset.map(map_to_dict)  # Apply mapping
train_dataset = train_dataset.batch(2)  # Reduce memory usage

In [6]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [7]:
# Define optimizer
optimizer = AdamWeightDecay(learning_rate=5e-5, weight_decay_rate=0.01)

# Define loss function (GPT-2 does NOT include loss by default)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=optimizer, loss=loss_fn)

In [8]:
# Train the model
model.fit(train_dataset, epochs=2)

# Save the fine-tuned model
model.save_pretrained("/mnt/data/fine_tuned_gpt2")
tokenizer.save_pretrained("/mnt/data/fine_tuned_gpt2")

print("Fine-tuning complete! Model saved.")

Epoch 1/2
365/365 [==============================] - 166s 375ms/step - loss: 0.1462
Epoch 2/2
365/365 [==============================] - 138s 378ms/step - loss: 0.0059
Fine-tuning complete! Model saved.


In [14]:
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")
print("Fine-tuning complete! Model saved.")


Fine-tuning complete! Model saved.


In [15]:
from google.colab import files
import shutil

# Zip the fine-tuned model
shutil.make_archive("fine_tuned_gpt2", 'zip', "/content/fine_tuned_gpt2")

# Download the zipped model
files.download("fine_tuned_gpt2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>